<h1>2b. Machine Learning using tf.estimator </h1>

In this notebook, we will create a machine learning model using tf.estimator and evaluate its performance.  The dataset is rather small (7700 samples), so we can do it all in-memory.  We will also simply pass the raw data in as-is. 

In [1]:
# import datalab.bigquery as bq
from google.cloud import bigquery as bq
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil

# print('Tensorflow Version:',tf.__version__)
print('google-cloud-bigquery Version:',bq.__version__)

google-cloud-bigquery Version: 1.5.0


/usr/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
client = bq.Client()

/usr/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/.
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/usr/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/.
  warnings.warn(_C

Read data created in the previous chapter.

In [3]:
# In CSV, label is the first column, after the features, followed by the key
CSV_COLUMNS = ['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']
FEATURES = CSV_COLUMNS[1:len(CSV_COLUMNS) - 1]
LABEL = CSV_COLUMNS[0]

df_train = pd.read_csv('./taxi-train.csv', header = None, names = CSV_COLUMNS)
df_valid = pd.read_csv('./taxi-valid.csv', header = None, names = CSV_COLUMNS)

In [4]:
df_train.head()

,fare_amount,pickuplon,pickuplat,dropofflon,dropofflat,passengers,key
0,12.0,-73.987625,40.750617,-73.971163,40.785180,1,0
1,4.5,-73.963620,40.774363,-73.953485,40.772665,1,1
2,4.5,-73.989649,40.756633,-73.985597,40.765662,1,2
3,10.0,-73.993950,40.727524,-74.006584,40.744240,1,3
4,2.5,-73.950223,40.668960,-73.948112,40.668872,6,4


In [5]:
df_train.columns.tolist()

['fare_amount',
 'pickuplon',
 'pickuplat',
 'dropofflon',
 'dropofflat',
 'passengers',
 'key']

In [6]:
FEATURES

['pickuplon', 'pickuplat', 'dropofflon', 'dropofflat', 'passengers']

<h2> Input function to read from Pandas Dataframe </h2>

In [7]:
def make_input_fn(df, num_epochs):
    return tf.estimator.inputs.pandas_input_fn(
            x = df,
            y = df[LABEL],
            batch_size = 128,
            num_epochs = num_epochs,
            shuffle = True,
            queue_capacity = 1000,
            num_threads = 1
          )

In [8]:
make_input_fn(df_train, num_epochs = 10)

<function tensorflow.python.estimator.inputs.pandas_io.pandas_input_fn.<locals>.input_fn()>

### Create feature columns for estimator

In [9]:
def make_feature_cols():
    input_columns = [tf.feature_column.numeric_column(k) for k in FEATURES]
    return input_columns

In [10]:
make_feature_cols()

[_NumericColumn(key='pickuplon', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='pickuplat', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='dropofflon', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='dropofflat', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='passengers', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

<h3> Linear Regression with tf.Estimator framework </h3>

In [18]:
tf.logging.set_verbosity(tf.logging.INFO)
# tf.logging.set_verbosity(tf.logging.WARN)

OUTDIR = 'taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # Delete an entire directory tree, and thus start fresh each time
# 也就是说'taxi_trained' directory 每次都会被删掉

model = tf.estimator.LinearRegressor(
      feature_columns = make_feature_cols(), model_dir = OUTDIR)

model.train(input_fn = make_input_fn(df_train, num_epochs = 10))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'taxi_trained', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f01e87bf080>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into taxi_trained/model.ckpt.
INFO:tensorflow:

Evaluate on the validation data (we should defer using the test data to after we have selected a final model).

In [19]:
def print_rmse(model, name, df):
    metrics = model.evaluate(input_fn = make_input_fn(df, 1))
    print('RMSE on {} dataset = {}'.format(name, np.sqrt(metrics['average_loss'])))
print_rmse(model, 'validation', df_valid)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-14-14:55:38
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-14-14:55:38
INFO:tensorflow:Saving dict for global step 608: average_loss = 109.465164, global_step = 608, label/mean = 11.666427, loss = 13018.536, prediction/mean = 10.865052
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 608: taxi_trained/model.ckpt-608
RMSE on validation dataset = 10.462559700012207


This is nowhere near our benchmark (RMSE of $6 or so on this data), but it serves to demonstrate what TensorFlow code looks like.  Let's use this model for prediction.

In [21]:
import itertools
# Read saved model and use it for prediction
model = tf.estimator.LinearRegressor(
      feature_columns = make_feature_cols(), model_dir = OUTDIR)
preds_iter = model.predict(input_fn = make_input_fn(df_valid, 1))
print('\nPredictions:\n',[pred['predictions'][0] for pred in list(itertools.islice(preds_iter, 5))])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'taxi_trained', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f01d3728b70>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Predictions:
 [10.88371, 10.822166, 10.818796, 10.822359, 10

This explains why the RMSE was so high -- the model essentially predicts the same amount for every trip.  Would a more complex model help? Let's try using a deep neural network.  The code to do this is quite straightforward as well.

<h3> Deep Neural Network regression </h3>

In [25]:
tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
model = tf.estimator.DNNRegressor(hidden_units = [32, 8, 2],
      feature_columns = make_feature_cols(), model_dir = OUTDIR)
model.train(input_fn = make_input_fn(df_train, num_epochs = 100));

print('--------------------------------------')
print_rmse(model, 'train', df_train)
print('--------------------------------------')
print_rmse(model, 'validation', df_valid)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'taxi_trained', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0218b5a5f8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into taxi_trained/model.ckpt.
INFO:tensorflow:

INFO:tensorflow:Saving dict for global step 6071: average_loss = 135.46214, global_step = 6071, label/mean = 11.666427, loss = 16110.319, prediction/mean = 6.5009227
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 6071: taxi_trained/model.ckpt-6071
RMSE on validation dataset = 11.63882064819336


We are not beating our benchmark with either model ... what's up?  Well, we may be using TensorFlow for Machine Learning, but we are not yet using it well.  That's what the rest of this course is about!

But, for the record, let's say we had to choose between the two models. We'd choose the one with the lower validation error. Finally, we'd measure the RMSE on the test data with this chosen model.

<h2> Benchmark dataset </h2>

Let's do this on the benchmark dataset.

In [87]:
import numpy as np
import pandas as pd

# import datalab.bigquery as bq
from google.cloud import bigquery as bq
client = bq.Client()


def create_query(phase, EVERY_N):
    """
    phase: 1 = train 2 = valid
    """
    base_query = """
    SELECT
      (tolls_amount + fare_amount) AS fare_amount,
      CONCAT(STRING(pickup_datetime), STRING(pickup_longitude), STRING(pickup_latitude), STRING(dropoff_latitude), STRING(dropoff_longitude)) AS key,
      DAYOFWEEK(pickup_datetime)*1.0 AS dayofweek,
      HOUR(pickup_datetime)*1.0 AS hourofday,
      pickup_longitude AS pickuplon,
      pickup_latitude AS pickuplat,
      dropoff_longitude AS dropofflon,
      dropoff_latitude AS dropofflat,
      passenger_count*1.0 AS passengers
    FROM
        `nyc-tlc.yellow.trips`
    WHERE
      trip_distance > 0
      AND fare_amount >= 2.5
      AND pickup_longitude > -78
      AND pickup_longitude < -70
      AND dropoff_longitude > -78
      AND dropoff_longitude < -70
      AND pickup_latitude > 37
      AND pickup_latitude < 45
      AND dropoff_latitude > 37
      AND dropoff_latitude < 45
      AND passenger_count > 0
    """

    if EVERY_N == None:
        if phase < 2:
            # Training
            query = "{0} AND ABS(HASH(pickup_datetime)) % 4 < 2".format(base_query)
        else:
            # Validation
            query = "{0} AND ABS(HASH(pickup_datetime)) % 4 == {1}".format(base_query, phase)
    else:
        query = "{0} AND ABS(HASH(pickup_datetime)) % {1} == {2}".format(base_query, EVERY_N, phase)
    
    return query

query = create_query(2, 100000)
for row in query:  # API request - fetches results
    # Row values can be accessed by field name or index
    print(row)

# df = bq.Query(query).to_dataframe()
# df = client.query(query).to_dataframe()
# .to_dataframe()

/usr/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/.
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/usr/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/.
  warnings.warn(_C



 
 
 
 
S
E
L
E
C
T


 
 
 
 
 
 
(
t
o
l
l
s
_
a
m
o
u
n
t
 
+
 
f
a
r
e
_
a
m
o
u
n
t
)
 
A
S
 
f
a
r
e
_
a
m
o
u
n
t
,


 
 
 
 
 
 
C
O
N
C
A
T
(
S
T
R
I
N
G
(
p
i
c
k
u
p
_
d
a
t
e
t
i
m
e
)
,
 
S
T
R
I
N
G
(
p
i
c
k
u
p
_
l
o
n
g
i
t
u
d
e
)
,
 
S
T
R
I
N
G
(
p
i
c
k
u
p
_
l
a
t
i
t
u
d
e
)
,
 
S
T
R
I
N
G
(
d
r
o
p
o
f
f
_
l
a
t
i
t
u
d
e
)
,
 
S
T
R
I
N
G
(
d
r
o
p
o
f
f
_
l
o
n
g
i
t
u
d
e
)
)
 
A
S
 
k
e
y
,


 
 
 
 
 
 
D
A
Y
O
F
W
E
E
K
(
p
i
c
k
u
p
_
d
a
t
e
t
i
m
e
)
*
1
.
0
 
A
S
 
d
a
y
o
f
w
e
e
k
,


 
 
 
 
 
 
H
O
U
R
(
p
i
c
k
u
p
_
d
a
t
e
t
i
m
e
)
*
1
.
0
 
A
S
 
h
o
u
r
o
f
d
a
y
,


 
 
 
 
 
 
p
i
c
k
u
p
_
l
o
n
g
i
t
u
d
e
 
A
S
 
p
i
c
k
u
p
l
o
n
,


 
 
 
 
 
 
p
i
c
k
u
p
_
l
a
t
i
t
u
d
e
 
A
S
 
p
i
c
k
u
p
l
a
t
,


 
 
 
 
 
 
d
r
o
p
o
f
f
_
l
o
n
g
i
t
u
d
e
 
A
S
 
d
r
o
p
o
f
f
l
o
n
,


 
 
 
 
 
 
d
r
o
p
o
f
f
_
l
a
t
i
t
u
d
e
 
A
S
 
d
r
o
p
o
f
f
l
a
t
,


 
 
 
 
 
 
p
a
s
s
e
n
g
e
r
_
c
o
u
n
t
*
1
.
0
 
A
S
 
p
a
s
s
e
n
g
e
r
s




In [85]:
from google.cloud import bigquery
client = bigquery.Client()

query = (
    '''
    SELECT extra FROM `nyc-tlc.yellow.trips`
    LIMIT 100
    ''')
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location='US')  # API request - starts the query

for row in query_job:  # API request - fetches results
    # Row values can be accessed by field name or index
    print(row)

/usr/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/.
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/usr/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/.
  warnings.warn(_C

Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), {'extra': 0})
Row((0.0,), 

In [15]:
print_rmse(model, 'benchmark', df)

NameError: name 'df' is not defined

RMSE on benchmark dataset is <b>9.41</b> (your results will vary because of random seeds).

This is not only way more than our original benchmark of 6.00, but it doesn't even beat our distance-based rule's RMSE of 8.02.

Fear not -- you have learned how to write a TensorFlow model, but not to do all the things that you will have to do to your ML model performant. We will do this in the next chapters. In this chapter though, we will get our TensorFlow model ready for these improvements.

In a software sense, the rest of the labs in this chapter will be about refactoring the code so that we can improve it.

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License